In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pickle
from sklearn.metrics import accuracy_score

In [2]:
model = tf.keras.models.load_model('./Saves/Seq/')
model.load_weights('./Saves/Seq/')

In [3]:
model_Xcept = tf.keras.models.load_model('./Saves/Xcept/')
model_Xcept.load_weights('./Saves/Xcept/')

In [4]:
model_ResNet50 = tf.keras.models.load_model('./Saves/Resnet/')
model_ResNet50.load_weights('./Saves/Resnet/')

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 38, 38, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 36, 36, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 36, 36, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 18, 18, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 6)                 9

In [6]:
model_Xcept.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [7]:
model_ResNet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [8]:
data_y=np.load('./Saves/data_y.npy')
data_x=np.load('./Saves/data_x.npy')
data_x_xcept = np.load('./Saves/data_x_xcept.npy')
data_y_xcept = np.load('./Saves/data_y_xcept.npy')

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X_train, X_test, Y_train, Y_test = train_test_split( data_x, data_y, test_size = 0.2, random_state = 10)
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [10]:

X_train_xcept, X_test_xcept, Y_train_xcept, Y_test_xcept = train_test_split( data_x_xcept, data_y_xcept, test_size = 0.2, random_state = 10 )
Y_train_xcept = to_categorical(Y_train_xcept)
Y_test_xcept = to_categorical(Y_test_xcept)

In [11]:
tf.keras.backend.clear_session()
loss,acc = model.evaluate(X_test, Y_test, verbose=2)
print("Restored sequential model, accuracy: {:5.2f}%".format(100*acc))

44/44 - 0s - loss: 0.4024 - accuracy: 0.9283
Restored sequential model, accuracy: 92.83%


In [12]:
tf.keras.backend.clear_session()
loss,acc = model_Xcept.evaluate(X_test_xcept, Y_test_xcept, verbose=2)
print("Restored Xception model, accuracy: {:5.2f}%".format(100*acc))

44/44 - 10s - loss: 0.0463 - accuracy: 0.9848
Restored Xception model, accuracy: 98.48%


In [13]:
tf.keras.backend.clear_session()
loss,acc = model_ResNet50.evaluate(X_test_xcept, Y_test_xcept, verbose=2)
print("Restored ResNet50 model, accuracy: {:5.2f}%".format(100*acc))

44/44 - 8s - loss: 0.8624 - accuracy: 0.7935
Restored ResNet50 model, accuracy: 79.35%


## Logistic Regression

In [14]:
from sklearn import linear_model
from sklearn.multiclass import OneVsOneClassifier
from sklearn import model_selection
from sklearn import metrics

from sklearn.model_selection import train_test_split


d1, d2, d3, d4 = data_x.shape
data_x_reshaped = data_x.reshape((d1, d2*d3*d4))

X_train_a, X_test_a, Y_train_a, Y_test_a = train_test_split(data_x_reshaped, data_y, test_size = 0.2, random_state = 17 )

In [15]:
with open("./Saves/logistic_save.sav", 'rb') as f:
    ovo = pickle.load(f)

In [16]:
lor_predictions =ovo.predict(X_test_a)
lor_predictions

array([0, 5, 2, ..., 4, 1, 3])

In [17]:
lor_accuracy = ovo.score(X_test_a,Y_test_a)
lor_accuracy

0.9072463768115943

In [18]:
cm = metrics.confusion_matrix(Y_test_a, lor_predictions)
cm

array([[213,   2,  10,   2,   3,   2],
       [  2, 205,   1,   8,   1,  11],
       [  5,   0, 229,   4,   8,   2],
       [  3,   9,   3, 208,   0,   0],
       [  6,   3,  10,   1, 201,   4],
       [  8,   9,   2,   6,   3, 196]], dtype=int64)

## Decision Tree

In [19]:
from sklearn.tree import DecisionTreeClassifier

In [20]:
with open("./Saves/decision_save.sav", 'rb') as f:
    dtree = pickle.load(f)

In [21]:
dtree_accuracy = dtree.score(X_test_a,Y_test_a)
print(dtree_accuracy)

0.6166666666666667


In [22]:
predictions_dtree = dtree.predict(X_test_a)

In [23]:
cm_dtree = metrics.confusion_matrix(predictions_dtree,Y_test_a)
print(cm_dtree)

[[148  16  13  14  12  11]
 [ 32 140  28  36  13  32]
 [ 17  17 152  26  29  17]
 [ 14  36  39 130  13  16]
 [  6   4   8   7 145  12]
 [ 15  15   8  10  13 136]]


## SVM (Support Vector Machine)

In [24]:
d1, d2, d3, d4 = data_x.shape 
data_x_reshaped = data_x.reshape((d1, d2*d3*d4))

In [25]:
from sklearn.model_selection import train_test_split

X_train_svm, X_test_svm, Y_train_svm, Y_test_svm = train_test_split(data_x_reshaped, data_y, test_size = 0.2, random_state = 25)

from sklearn.svm import SVC

In [26]:
with open("./Saves/gcv.sav", 'rb') as f:
    grids = pickle.load(f)

In [27]:
print(grids.best_params_)

print(grids.best_score_)

{'C': 10, 'kernel': 'rbf', 'tol': 0.1}
0.9190217391304347


In [28]:
with open("./Saves/svm_save.sav", 'rb') as f:
    svc = pickle.load(f)

In [29]:
svm_accuracy = svc.score(X_test_svm,Y_test_svm)
print(svm_accuracy)

0.936231884057971


In [30]:
predictions_svm = svc.predict(X_test_svm)
cm_svm = metrics.confusion_matrix(predictions_svm,Y_test_svm) 
print(cm_svm)

[[219   5   4   1   2   5]
 [  1 213   4   4   0   2]
 [  5   1 213   3   8   5]
 [  1   4   4 223   1   4]
 [  3   2   3   1 218   2]
 [  2   5   1   2   3 206]]
